In [141]:
from srwr.srwr import SRWR
import pickle as p
from collections import defaultdict
def generateTSVfromDict(global_dict, query_2gram):
    namelist = []
    output_str = ''
    name_dict = defaultdict(int)
    contradictory_dict = defaultdict(list)
    threshold=THRES
    hasDistance = False
    if type(query_2gram[0][0][0]) == type((1,2)):
        hasDistance = True
    # add functions as random variables
    for key in global_dict:
        idx_begin = len(namelist)
        idx_end = idx_begin + len(global_dict[key])
        for ((predict, libraryname), score) in global_dict[key]:
        #for (predict, libraryname) in global_dict[key]:
            name = key + '##' + predict
            name = key + '##' + predict
            name_dict[name] = len(namelist)
            contradictory_dict[predict].append(len(namelist))
            namelist.append(name)


#         for i in range(idx_begin, idx_end):
#             for j in range(i + 1, idx_end):
#                 output_str += str(i) + '\t' + str(j) + '\t-10\n'
    #p.dump(global_dict, open('global_dict','w'))
    for key in contradictory_dict:
        contradicts = contradictory_dict[key]
        if len(contradicts) > 1:
            for i in range(len(contradicts)):
                for j in range(i + 1, len(contradicts)):
                    output_str += str(contradicts[i]) + '\t' + str(contradicts[j]) + '\t-30\n'


    for i in query_2gram:
        if i[2] > threshold:
            if not hasDistance:
                test_src = i[0][0]
                test_des = i[0][1]
            else:
                test_src = i[0][0][0]
                test_des = i[0][0][1]
            if test_src == test_des:
                continue
            # if len(i[1]) > 200:
            #     continue
            if test_src not in global_dict or test_des not in global_dict:
                continue
          
            src_list = global_dict[test_src]
            des_list = global_dict[test_des]
            funcList = i[1]
            
            for predicted_func in funcList:
                if hasDistance:
                    querydistance = i[0][1]
                    resultdistance = predicted_func[3]
                    if querydistance != resultdistance:
                        continue

                src = predicted_func[2][0]
                des = predicted_func[2][1]
                libraryName = predicted_func[0]
                if test_src + '##' + src in namelist and test_des + '##' + des in namelist:
                    src_ind = name_dict[test_src + '##' + src]
                    des_ind = name_dict[test_des + '##' + des]
                    output_str += str(src_ind) + '\t' + str(des_ind) + '\t1\n'
                else:
                    continue

    with open('functioncallgraph.tsv', 'w') as f:
        f.write(output_str)
    p.dump(namelist, open('namelist', 'w'))
    #return output_str

def analyse(namelist, rd):
    tp = []
    fp = []
    prediction = defaultdict(list)
    for i, d in enumerate(rd):
        name = namelist[i].split('##')[0]
        predict = namelist[i].split('##')[1]
        if d >= 0:
            prediction[name].append([predict, d])
    
    for name in prediction.keys():
        sorted(prediction[name], key=lambda x: x[1], reverse=True)
        maxscore = prediction[name][0][1]
        finalpredict = []
        for [predict, score] in prediction[name]:
            if score < maxscore:
                break
            finalpredict.append(predict)
        if name in finalpredict:
            tp.append(name)
        else:
            fp.append(name)
    return tp, fp, prediction


In [142]:
THRES = 0.9
global_dict = p.load(open('prediction', 'r'))
query_2gram = p.load(open('/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/test_kNN_0924_2gram.p', 'r'))
generateTSVfromDict(global_dict, query_2gram)
srwr = SRWR()
input_path = 'functioncallgraph.tsv'
srwr.read_graph(input_path)
srwr.normalize()

In [143]:
sure_dict = p.load(open('sure_dict', 'r'))
seeds = []
namelist = p.load(open('namelist', 'r'))
for key in sure_dict.keys():
    prediction_set = set(sure_dict[key])
    if len(prediction_set) == 1:
        idx = namelist.index(key + '##' + list(prediction_set)[0][0])
        seeds.append(idx)
print seeds

[4181, 1233, 2344, 19, 413, 1370, 1371, 287, 1380, 2819, 9, 11, 12, 13, 1387, 1389, 1390, 16, 887, 17, 1401, 189, 2834, 211, 2838, 23, 1431, 25, 26, 27, 4971, 3029, 41, 4261, 81, 1459, 2833, 1463, 1464, 3698, 1469, 65, 2818, 4292, 4293, 1475, 3278, 70, 71, 1157, 4301, 4975, 1480, 1481, 76, 1482, 1484, 80, 4310, 4313, 1492, 4315, 1493, 1495, 4319, 1497, 117, 4321, 4322, 119, 2882, 721, 124, 126, 2885, 1515, 1517, 4386, 1526, 134, 1529, 2896, 149, 150, 2908, 1045, 164, 4413, 2916, 166, 1549, 1550, 970, 167, 168, 1553, 4424, 176, 3950, 183, 1613, 2941, 1616, 3576, 3720, 4438, 1217, 1648, 190, 193, 2952, 1653, 1654, 2645, 201, 1658, 204, 205, 1660, 4452, 4453, 2968, 3962, 1665, 4460, 234, 2839, 3000, 2987, 1685, 3237, 1696, 1697, 4489, 1699, 1700, 265, 4497, 270, 273, 1707, 274, 3004, 293, 4423, 276, 277, 278, 5290, 3023, 4510, 3026, 4511, 3028, 1082, 5304, 4516, 4528, 1750, 3069, 3070, 1756, 1757, 3072, 4535, 954, 306, 310, 312, 1780, 320, 1788, 4582, 2142, 3090, 3093, 322, 3094, 3095, 94

In [70]:
def union_dict(dict1, dict2):
    return (dict2.update(dict1))
prediction_total = defaultdict(list)
for seed in seeds:
    rd, rp, rn, residuals = srwr.query(seed)
    namelist = p.load(open('namelist', 'r'))
    tp, fp, prediction = analyse(namelist, rd)
    union_dict(prediction_total, prediction)
print prediction_total

SRWR scores have converged: 100%|██████████| 300/300 [00:00<00:00, 451.93it/s]


SRWR scores have converged: 100%|██████████| 300/300 [00:00<00:00, 531.99it/s]


SRWR scores have converged: 100%|██████████| 300/300 [00:00<00:00, 647.96it/s]


SRWR scores have converged: 100%|██████████| 300/300 [00:00<00:00, 451.73it/s]


SRWR scores have converged: 100%|██████████| 300/300 [00:00<00:00, 647.73it/s]


defaultdict(<type 'list'>, {})


In [144]:
namelist = p.load(open('namelist', 'r'))
rd, rp, rn, residuals = srwr.query(seeds[0])
tp1, fp1, prediction1 = analyse(namelist, rd)
rd, rp, rn, residuals = srwr.query(seeds[100])
tp2, fp2, prediction2 = analyse(namelist, rd)
rd, rp, rn, residuals = srwr.query(seeds[200])
tp3, fp3, prediction3 = analyse(namelist, rd)

SRWR scores have converged: 100%|██████████| 300/300 [00:00<00:00, 1410.96it/s]


In [145]:
set1 = set(prediction1.keys())
set2 = set(prediction2.keys())
set3 = set(prediction3.keys())
print len(set1), len(set2), len(set1|set2)
print len(tp1), len(tp2), len(set(tp1)|set(tp2))
print len(fp1), len(fp2), len(set(fp1)|set(fp2))

2079 2082 2110
1599 1609 1612
480 473 506


In [113]:
p.dump(set(fp1)|set(fp2), open('fp', 'w'))

In [28]:
namelist[:100]

['ngx_http_parse_chunked##PKCS5_PBKDF2_HMAC',
 'ngx_http_parse_chunked##BN_mod_exp2_mont',
 'ngx_http_parse_chunked##BN_mod_exp_mont',
 'ngx_http_parse_chunked##ssl3_write_bytes',
 'ngx_http_proxy_pass.part.4##PKCS12_setup_mac',
 'ngx_http_proxy_pass.part.4##PKCS7_SIGNER_INFO_set',
 'read_string_inner##read_string_inner',
 'read_string_inner##EVP_BytesToKey',
 'read_string_inner##dtls1_do_write',
 'RSA_padding_check_PKCS1_type_2##RSA_padding_check_PKCS1_type_2',
 'RSA_padding_check_PKCS1_type_2##RSA_padding_check_PKCS1_type_1',
 'RSA_padding_check_PKCS1_type_2##CRYPTO_malloc',
 'engine_cleanup_add_first##CRYPTO_malloc',
 'engine_cleanup_add_first##ERR_put_error',
 'engine_cleanup_add_first##engine_cleanup_add_last',
 'engine_cleanup_add_first##engine_cleanup_add_first',
 'RSA_padding_check_PKCS1_type_1##RSA_padding_check_PKCS1_type_1',
 'RSA_padding_check_PKCS1_type_1##RSA_padding_check_PKCS1_type_2',
 'RSA_padding_check_PKCS1_type_1##BN_sub',
 'PEM_read_bio_Parameters##dtls1_buffer_re

In [33]:
sure_dict = p.load(open('sure_dict','r'))
print sure_dict

defaultdict(<type 'list'>, {'X509V3_NAME_from_section': [('X509V3_NAME_from_section', 'libcrypto')], 'ECDH_compute_key': [('ECDH_compute_key', 'libcrypto')], 'sk_sort': [('sk_sort', 'libcrypto')], 'PKCS7_simple_smimecap': [('PKCS7_simple_smimecap', 'libcrypto')], 'trust_1oid': [('trust_1oid', 'libcrypto')], 'SHA224_Final': [('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcrypto'), ('SHA256_Final', 'libcr

In [44]:
namelist.index('RSA_padding_check_PKCS1_OAEP##RSA_padding_check_PKCS1_OAEP')

14873